In [1]:
from src.data.dataset_functions import make_dataframe_from_rawcsv,provide_rawcsv

provide_rawcsv(True)
new_dataframe = make_dataframe_from_rawcsv()

providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1d5ece2a-c1b1-4c33-848f-c4840596a3b7;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 119ms :: artifacts dl 8ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------

In [2]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from src.data.dataset_functions import transform_dataframe_colums,transform_dataframe_normalize_seconds,transform_dataframe_timestamp

def transform_dataframe(df_input: DataFrame) -> DataFrame:
    ''' Transforms a dataframe from its source structure into something more usable '''
    df_transform = transform_dataframe_filter_nomods(df_input)
    df_transform = transform_dataframe_timestamp(df_transform)
    df_transform = transform_dataframe_normalize_seconds(df_transform)
    df_transform = transform_dataframe_colums(df_transform)
    return df_transform

def transform_dataframe_filter_nomods(df_input: DataFrame) -> DataFrame:
    ''' filters mod data - they have 2 coordinate pairs per row '''
    df_output = df_input.where(F.size(F.split('coordinate', ',')) == 2)
    return df_output

def transform_dataframe_filter_onlymods(df_input: DataFrame) -> DataFrame:
    ''' outputs only mod data - 2 coordinate pairs '''
    df_output = df_input.where(F.size(F.split('coordinate', ',')) > 2)
    return df_output

def transform_dataframe_modcolumns(df_input: DataFrame) -> DataFrame:
    ''' Transforms columns from [\'timestamp\',\'user_id\',\'pixel_color\',\'coordinate\']
    into [\'user_id\',\'x1\',\'y1\',\'x2\',\'y2\',\'t\',\'pixel_color\']'''
    df_output = df_input.select('user_id',
                                F.split('coordinate', ',').getItem(
                                    0).cast('int').alias('x1'),
                                F.split('coordinate', ',').getItem(
                                    1).cast('int').alias('y1'),
                                F.split('coordinate', ',').getItem(
                                    2).cast('int').alias('x2'),
                                F.split('coordinate', ',').getItem(
                                    3).cast('int').alias('y2'),
                                F.col('timestamp').alias('t'),
                                'pixel_color')
    return df_output

def transform_dataframe_onlymods(df_input: DataFrame) -> DataFrame:
    ''' Transfroms a dataframe from its source structure but uses only the rows with moderator data '''
    df_transform = transform_dataframe_filter_onlymods(df_input)
    df_transform = transform_dataframe_timestamp(df_transform)
    df_transform = transform_dataframe_normalize_seconds(df_transform)
    df_transform = transform_dataframe_modcolumns(df_transform)
    return df_transform


In [3]:

transformed_dataframe = transform_dataframe(new_dataframe)

In [4]:
mod_dataframe = transform_dataframe_onlymods(new_dataframe)

In [5]:
mod_dataframe.show()

+-------+---+---+---+---+---+-----------+
|user_id| x1| y1| x2| y2|  t|pixel_color|
+-------+---+---+---+---+---+-----------+
+-------+---+---+---+---+---+-----------+

